In [1]:
import os
import cv2
import pickle
import imageio
import numpy as np
def psnr(img1, img2):
    mse = np.mean((img1 - img2) ** 2.0)
    if mse == 0:
        return 100.
    PIXEL_MAX = 255.0
    return 20 * np.log10(PIXEL_MAX / np.sqrt(mse))

def unpickle(file):
    with open(file, 'rb') as fo:
        dict = pickle.load(fo, encoding='bytes')
    return dict

import torch

In [58]:
###  Load the data
path = f'E:/PHD_DOC/RES1_SemanticCommunication5/Code/Benchmark/test_bpg_response/cifar-10-batches-py'
train_data = np.empty((50000, 32, 32, 3), dtype=np.uint8)
train_labels = np.empty(50000, dtype=np.uint8)
for i in range(0, 5):
    data_train = unpickle(os.path.join(path, 'data_batch_{}'.format(i+1)))
    train_data[i*10000:(i+1)*10000] = data_train[b'data'].reshape(-1, 3, 32, 32).transpose((0, 2, 3, 1))

data_test = unpickle(os.path.join(path, 'test_batch'))
test_data = data_test[b'data'].reshape(-1, 3, 32, 32).transpose((0, 2, 3, 1))

### Save the image to image list
train_save_dir =  './cifar10_list/cifar10_train'
test_save_dir = './cifar10_list/cifar10_test'
if not os.path.exists(train_save_dir):
    os.makedirs(train_save_dir)
if not os.path.exists(test_save_dir):
    os.makedirs(test_save_dir)
    
train_num = train_data.shape[0]
for i in range(train_num):
    path_name = f'{train_save_dir}/cifar-train-{i:05d}.png'
    imageio.imwrite(path_name,train_data[i])
    if i%500==0:
        print('Finish %d train images ' % (i))
    
test_num = test_data.shape[0]
for i in range(test_num):
    path_name = f'{test_save_dir}/cifar-test-{i:05d}.png'
    imageio.imwrite(path_name, test_data[i])
    if i%500==0:
        print('Finish %d test images ' % (i))

KeyboardInterrupt: 

In [87]:
code_rate = 1/2.
M = 4
num_pixel = 32*32*3
cbr = 1/12.

length = num_pixel * cbr * M * code_rate / 8
print(length)
print(length+7)

64.0
71.0


In [2]:
# Replace 'input_image.png' and 'compressed_image.bpg' with your file paths
base_dir = 'E:/PHD_DOC/RES1_SemanticCommunication5/Code/Benchmark/test_bpg_response'
test_comp_dir = './cifar10_list/cifar10_test_compress'
if not os.path.exists(test_comp_dir):
    os.makedirs(test_comp_dir)
test_bits_dir = './cifar10_list/cifar10_bitstream_compress'
if not os.path.exists(test_bits_dir):
    os.makedirs(test_bits_dir)
    
aver_len = 0
aver_psnr = 0
test_exp_num = 200
for i in range(test_exp_num):
    # index = np.random.randint(1, 10000)
    ######  编码方式修改为 cv2.imwrite('saved_image.jpg',image,[cv2.IMWRITE_JPEG_QUALITY,95])，类似下面 -q的值  以支持JPEG  JPEG2000等方式
    index = i+10
    input_image_path = f'{base_dir}/cifar10_list/cifar10_test/cifar-test-{index:05d}.png'
    bit_stream_path = f'{base_dir}/cifar10_list/cifar10_bitstream_compress/cifar-test-{index:05d}.txt'
    
    # Encode the input image to BPG QP 0~50
    encode_command = f'{base_dir}/bpg_lib/bpgenc -e x265 -q 39 -f 444 -o {bit_stream_path}  {input_image_path}'
    os.system(encode_command)

    # Decode the BPG image back to PNG
    decoded_image_path = f'{base_dir}/cifar10_list/cifar10_test_compress/cifar-test-compress-{index:05d}.png'
    decode_command = f'{base_dir}/bpg_lib/bpgdec -o {decoded_image_path} {bit_stream_path}'
    os.system(decode_command)

    # 打开一个文件（如果文件不存在，将创建一个新文件）
    file_path = bit_stream_path
    file = open(file_path, "rb")  # 使用 "r" 模式表示只读模式  读取文件的内容
    file_contents = file.read()
    # print(((file_contents[0:7])))
    aver_len = aver_len + len(file_contents)
    file.close()
    
    img1 = cv2.imread(input_image_path)
    img2 = cv2.imread(decoded_image_path)
    # print(psnr(img1 + 0., img2 + 0.))
    aver_psnr = aver_psnr + psnr(img1 + 0., img2 + 0.)
    # print(aver_psnr)
    if i % 20==0:
        print('Average length:{:.2f}'.format(aver_len/(i+1)))
        print('Average PSNR:{:.2f}'.format(aver_psnr/(i+1)))
print('Average length:{}'.format(aver_len/test_exp_num))
print('Average PSNR:{}'.format(aver_psnr/test_exp_num ))

Average length:108.00
Average PSNR:29.20
Average length:141.05
Average PSNR:26.85
Average length:136.15
Average PSNR:26.84
Average length:134.25
Average PSNR:26.94
Average length:131.02
Average PSNR:27.17
Average length:131.59
Average PSNR:27.17
Average length:129.75
Average PSNR:27.25
Average length:131.08
Average PSNR:27.18
Average length:130.04
Average PSNR:27.23
Average length:129.87
Average PSNR:27.25
Average length:129.84
Average PSNR:27.25469430130117


In [ ]:
for snr in range(20):
    SNRdB = snr  #dB
    SNR = 10**(SNRdB/10.)
    C = np.log2(SNR+1)
    CBR = 1./16
    L1 = 512*768*3 * CBR 
    length = L1 * C / 8.
    print('SNR is %.2f, C is %.2f bits, length is %.2f' %(SNRdB, C, length))

for ratio in [1/12., 1/6., 1/4., 1/3., 1/2.]:
    print(32*32*3. * ratio * 4. *2./3. /8.)

In [9]:
####    Multi-hop  scenario
aver_len = 0
aver_psnr = 0
tx_time = 10
aver_len_group = np.zeros(tx_time) 
test_exp_num = 100
for i in range(test_exp_num):
    # index = np.random.randint(1, 10000)
    index = i+1000
    input_image_path = f'{base_dir}/cifar10_list/cifar10_test/cifar-test-{index:05d}.png'
    bit_stream_path = f'{base_dir}/cifar10_list/cifar10_bitstream_compress/cifar-test-{index:05d}.txt'
    
    # Encode the input image to BPG QP 0~50
    encode_command = f'{base_dir}/bpg_lib/bpgenc -e x265 -q 39 -f 444 -o {bit_stream_path}  {input_image_path}'
    os.system(encode_command)

    # Decode the BPG image back to PNG
    for n in range(tx_time):
        decoded_image_path = f'{base_dir}/cifar10_list/cifar10_test_compress/cifar-test-compress-{index:05d}.png'
        decode_command = f'{base_dir}/bpg_lib/bpgdec -o {decoded_image_path} {bit_stream_path}'
        os.system(decode_command)

        # 打开一个文件（如果文件不存在，将创建一个新文件）
        file_path = bit_stream_path
        file = open(file_path, "rb")  # 使用 "r" 模式表示只读模式  读取文件的内容
        file_contents = file.read()
        # print(len(file_contents))
        
        file.close()
        encode_command = f'{base_dir}/bpg_lib/bpgenc -e x265 -q 39 -f 444 -o {bit_stream_path}  {decoded_image_path}'
        os.system(encode_command)
        img1 = cv2.imread(input_image_path)
        img2 = cv2.imread(decoded_image_path)
        # print(psnr(img1 + 0., img2 + 0.))
        aver_len = aver_len + len(file_contents)
        # aver_psnr = aver_psnr + psnr(img1 + 0., img2 + 0.)
        aver_len_group[n] += psnr(img1 + 0., img2 + 0.)
    # print(aver_psnr)
    aver_psnr = aver_psnr + psnr(img1 + 0., img2 + 0.)

    if i % 20==0:
        print((aver_len_group/(i+1)))
torch.set_printoptions(precision=2)
print(torch.tensor(aver_len_group/test_exp_num))


[26.47240806 26.47240806 26.47240806 26.47240806 26.47240806 26.47240806
 26.47240806 26.47240806 26.47240806 26.47240806]
[27.30039959 26.95220403 26.85053881 26.82690187 26.81743971 26.80991578
 26.80779592 26.80541279 26.80541279 26.80541279]
[27.09574641 26.69617396 26.56815263 26.54482211 26.529822   26.51307145
 26.50947847 26.50797668 26.50825784 26.50797668]
[27.1327256  26.69088092 26.56286672 26.5297047  26.51260356 26.50134499
 26.49893003 26.49792063 26.49810961 26.49792063]
[27.20968453 26.77772546 26.65355459 26.62660379 26.61216971 26.60369104
 26.60187237 26.6011122  26.60125452 26.6011122 ]
tensor([27.19, 26.77, 26.63, 26.60, 26.57, 26.57, 26.57, 26.57, 26.57, 26.57],
       dtype=torch.float64)
